# XML Streaming

#### Model

In [3]:
from openai import AzureOpenAI
from dotenv import load_dotenv
from os import getenv


load_dotenv()

True

In [7]:
client = AzureOpenAI(
    api_key=getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=getenv("AZURE_OPENAI_API_VERSION")
)
MODEL_NAME = "gpt-4o"

#### XML Parser

In [5]:
from langchain_core.output_parsers import XMLOutputParser

#### Sample Prompt

In [6]:
SAMPLE_PROMPT = """
You are an expert data translation tool. Your task is to follow given instruction about a {topic} and generated appropriate response in XML format.

Example:
Input: List out the features of the employee named Joe aged 24 who was born on 27th june 1990
Output:
<employee>
 <name>Joe</name>
 <age>24</age>
 <dob>27-06-1990</dob>
 </employee>
"""

In [15]:
query = "A basic summary of Keanu Reeves"

messages = [
    {
        "role": "system", "content": SAMPLE_PROMPT
    },
    {
        "role": "user", "content": query
    }
]

In [ ]:
completion = client.chat.completions.create(
  model=MODEL_NAME,
  store=False,
  messages=messages
)

print(completion.choices[0].message.content)

```xml
<celebrity>
    <name>Keanu Reeves</name>
    <profession>Actor</profession>
    <nationality>Canadian</nationality>
    <birthdate>02-09-1964</birthdate>
    <notableWorks>
        <work>The Matrix Series</work>
        <work>John Wick Series</work>
        <work>Speed</work>
        <work>Bill & Ted's Excellent Adventure</work>
    </notableWorks>
    <characteristics>
        <trait>Humble personality</trait>
        <trait>Philanthropic efforts</trait>
        <trait>Action movie stardom</trait>
    </characteristics>
</celebrity>
```


#### XML Tree Parsing

In [18]:
import xml.etree.ElementTree as ET

In [ ]:
# Define the prompt with the XML schema
prompt = """
Generate XML content for the given query
Example:
<root>
  <tag1>value1</tag1>
  <tag2>value2</tag2>
  <tag3>value3</tag3>
</root>
Ensure the output is valid XML, with properly closed tags and no extraneous content. Provide the content for tag1, tag2, and tag3 based on the context: "Describe a book with title, author, and genre."
Do not include ` or codeblocks
"""

# Start streaming
xml_output = ""

user_input = "Poodle dog age 3 healthy" # this will be from text area

stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that generates valid XML according to the provided schema."},
        {"role": "user", "content": prompt},
        {"role": "user", "content": user_input}
    ],
    stream=True
)

# Process streamed chunks

for chunk in stream:
    if chunk.choices and chunk.choices[0].delta:
        data = chunk.choices[0].delta
        if data and hasattr(data, "content"):
          content = data.content
          if content == None:
            continue
          xml_output += content
          
          print(content, end="", flush=True)

# Close the stream
stream.close()


<root>
  <tag1>Theroot>
  <tag1>The Great Gatsby</tag1>
  <tag2>F. Scott Fitzgerald</tag2>
  <tag3>Classic Fiction</tag3>
</root>